
# Tables 1, 2, and 3

In [1]:
from functools import partial
from multiprocess import Pool, cpu_count, current_process
import pandas as pd
from tqdm.notebook import tqdm

from pref_voting.generate_weighted_majority_graphs import *
from pref_voting.voting_methods import *

from pref_voting.generate_profiles import *

# needed to ensure that random numbers are different in each process (for multiprocessing)
from os import register_at_fork
register_at_fork(after_in_child=np.random.seed)


In [2]:
## Condorcetian Candidates

def is_condorcetian(vm, cand, edata): 
    """
    Definition 4.4: Return True if cand is Condorcetian in edata according to vm
    """
    candidates = edata.candidates
    
    for c in edata.dominates(cand): # cand is majority preferred to c
        ws_minus = vm(edata, curr_cands = [_c for _c in candidates if _c != c])
        if cand in ws_minus: 
            return True
    return False

        
def is_weak_condorcetian(vm, cand, edata): 
    """
    Definition 4.4: Return True if cand is weak Condorcetian in edata according to vm
    """
    
    candidates = edata.candidates
    
    for c in edata.candidates:
        if edata.majority_prefers(cand, c) or edata.is_tied(cand, c):
            ws_minus = vm(edata, curr_cands = [_c for _c in candidates if _c != c])
            if cand in ws_minus: 
                return True        
    return False



## Table 1

Of four-candidate linear profiles in which $BP(\mathbf{P}) \neq SC(\mathbf{P})$, find the percentage in which Beat Path violates strong stability for winners, i.e., there are $a,b\in X(\mathbf{P})$ such that $a\in BP(\mathbf{P}_{-b})$, $Margin_\mathbf{P}(a, b) \geq 0$, and $a\notin BP(\mathbf{P})$

In [3]:
df = pd.read_csv("./tables-data/violations_ssw.csv")
df = pd.read_csv("./violations_ssw.csv")

In [4]:
df

,vm,comparison_vm,num_cands,num_voters,num_profiles,probmod,perc_diff_ws,perc_ssw_violation,perc_ssw_violation_cond_diff_ws
0,Beat Path,Split Cycle,4,"(10, 11)",2000000,IC,0.005159,0.005148,0.997868
1,Beat Path,Split Cycle,4,"(20, 21)",2000000,IC,0.008280,0.008157,0.985085
2,Beat Path,Split Cycle,4,"(50, 51)",2000000,IC,0.010549,0.010040,0.951749
3,Beat Path,Split Cycle,4,"(100, 101)",2000000,IC,0.011314,0.010560,0.933316
4,Beat Path,Split Cycle,4,"(500, 501)",2000000,IC,0.012127,0.010996,0.906778
5,Beat Path,Split Cycle,4,"(1000, 1001)",2000000,IC,0.012480,0.011239,0.900521
6,Beat Path,Split Cycle,4,"(5000, 5001)",2000000,IC,0.012315,0.011045,0.896914
7,Beat Path,Split Cycle,4,"(10, 11)",2000000,IAC,0.005325,0.005313,0.997746
8,Beat Path,Split Cycle,4,"(20, 21)",2000000,IAC,0.008195,0.007975,0.973034
9,Beat Path,Split Cycle,4,"(50, 51)",2000000,IAC,0.009745,0.009069,0.930631


In [5]:
print("\nOf the profiles in which the Beat Path and Split Cycle winners are different, the percent of strong stability for winners violations for Beat Path (Table 1).")
df_table = df.pivot(index="probmod", columns="num_voters", values="perc_ssw_violation_cond_diff_ws")
df_table


Of the profiles in which the Beat Path and Split Cycle winners are different, the percent of strong stability for winners violations for Beat Path (Table 1).


num_voters,"(10, 11)","(100, 101)","(1000, 1001)","(20, 21)","(50, 51)","(500, 501)","(5000, 5001)"
probmod,,,,,,,
IAC,0.997746,0.912119,0.897506,0.973034,0.930631,0.899583,0.895494
IC,0.997868,0.933316,0.900521,0.985085,0.951749,0.906778,0.896914
MALLOWS-RELPHI-R,0.998290,0.941248,0.889693,0.989275,0.966455,0.910811,0.895683
URN-R,0.991946,0.918287,0.899818,0.969054,0.926678,0.907615,0.897029


In [6]:
print("\nThe percent of strong stability for winners violations for Beat Path.")

df_table = df.pivot(index="probmod", columns="num_voters", values="perc_ssw_violation")
df_table


The percent of strong stability for winners violations for Beat Path.


num_voters,"(10, 11)","(100, 101)","(1000, 1001)","(20, 21)","(50, 51)","(500, 501)","(5000, 5001)"
probmod,,,,,,,
IAC,0.005313,0.009502,0.009680,0.007975,0.009069,0.009595,0.009588
IC,0.005148,0.010560,0.011239,0.008157,0.010040,0.010996,0.011045
MALLOWS-RELPHI-R,0.001167,0.000777,0.000246,0.001337,0.001066,0.000337,0.000124
URN-R,0.002217,0.003827,0.003961,0.003241,0.003652,0.003886,0.003925


In [7]:
print("\nThe percent of profiles in which Beat Path and Split Cycle have different winners.")
df_table = df.pivot(index="probmod", columns="num_voters", values="perc_diff_ws")
df_table


The percent of profiles in which Beat Path and Split Cycle have different winners.


num_voters,"(10, 11)","(100, 101)","(1000, 1001)","(20, 21)","(50, 51)","(500, 501)","(5000, 5001)"
probmod,,,,,,,
IAC,0.005325,0.010417,0.010786,0.008195,0.009745,0.010665,0.010708
IC,0.005159,0.011314,0.012480,0.008280,0.010549,0.012127,0.012315
MALLOWS-RELPHI-R,0.001169,0.000825,0.000276,0.001352,0.001103,0.000370,0.000139
URN-R,0.002235,0.004167,0.004402,0.003344,0.003941,0.004281,0.004375


In [8]:

def has_strong_stability_for_winners_violation(vm, edata, weak_condorcetian_candidates = None): 
    """
    Definition 4.2.3: Return True if there are candidates a and b such that
      1. a in a winner according to vm in edata without b in the election, 
      2. a is majority preferred to b or tied with b, and 
      3. a is not a winner in edata
    
    This is equivalent to requiring that all weak Condorcetian candidates are winners in edata according to vm. 
    """
    candidates = edata.candidates
    
    if weak_condorcetian_candidates is None: 
        weak_condorcetian_candidates = [c for c in candidates if is_weak_condorcetian(vm, c, edata)]
    
    ws = vm(edata)
        
    return not set(weak_condorcetian_candidates).issubset(set(ws))



In [9]:
def record_strong_stability_for_winners_violation(vms, main_vm, num_cands, num_voters, probmod,  t): 
    
    prof = generate_profile(num_cands, num_voters, probmod=probmod)
    
    data = {vm.name: dict() for vm in vms}
    for vm in vms: 
        weak_condorcetian_candidates = [c for c in prof.candidates if is_weak_condorcetian(vm, c, prof)]
        
        main_ws = main_vm(prof)
        vm_ws = vm(prof)
        data[vm.name] =  {
            "diff_ws": main_ws != vm_ws, 
            "ssw_violation": has_strong_stability_for_winners_violation(vm, prof, weak_condorcetian_candidates = weak_condorcetian_candidates)
        }
    return data
    

In [12]:
## Parameters for the simulation

# WARNING: The simulation takes a very long time!  
# Set the following to True to skip running the simulation
SKIP_SIMULATION = True  

num_trials = 1_000_000 # WARNING: This takes a very long time to run!

numbers_of_candidates = [
#     3,
    4,
#     5,
#     6,
#     7,
#     8,
#     10, 
#     15, 
#     20
]

numbers_of_voters = [
    (10, 11),
    (20, 21),
    (50, 51),
    (100, 101),
    (500, 501),
    (1000, 1001),
    (5000, 5001),
] 

# uncomment probability models as needed
prob_models = [
    "IC", 
    "IAC", 
    "MALLOWS-RELPHI-R", 
    "URN-R"
]
 
vms = [
    beat_path,
#     llull,
#     copeland,
#     minimax,
#     uc_gill, # uncovered set
#     instant_runoff,  
#     borda,
#     plurality,
#     getcha
]

main_vm = split_cycle

# use parallel processing to speed up the search
cpus = cpu_count()
print(f'CPUS: {cpus}')
pool = Pool(10) # set to cpus when you can use all the processors

print(f"SKIP_SIMULATION is set to {SKIP_SIMULATION}")

CPUS: 12
SKIP_SIMULATION is set to True


In [25]:
%%time 

data_for_df = {
    "vm": list(),
    "comparison_vm": list(),
    "num_cands": list(), 
    "num_voters": list(),
    "num_profiles": list(),
    "probmod": list(),
    "perc_diff_ws": list(),
    "perc_ssw_violation": list(),
    "perc_ssw_violation_cond_diff_ws": list()
}

for probmod in prob_models:

    print(probmod)

    voting_scenarios = list(product(numbers_of_candidates, numbers_of_voters))

    data = {vm.name: {_vs: {"total_profiles": 0, 
                            "total_even": list(),
                            "total_odd": list()} 
                      for _vs in voting_scenarios} 
            for vm in vms}

    for vs in voting_scenarios:
        print(f"     {vs}")
        num_candidates = vs[0]
        num_voters_even = vs[1][0]
        num_voters_odd = vs[1][1]

        # find the violations in parallel

        record_violations_even = partial(
            record_strong_stability_for_winners_violation,
            vms, 
            main_vm, 
            num_candidates, 
            num_voters_even, 
            probmod)

        record_violations_odd = partial(
            record_strong_stability_for_winners_violation,
            vms, 
            main_vm, 
            num_candidates, 
            num_voters_odd, 
            probmod)

        violations_even = pool.map(record_violations_even, range(num_trials))
        violations_odd = pool.map(record_violations_odd, range(num_trials))

        # done calculating the violations
        for vm in vms:
            data_even = [d[vm.name] for d in violations_even]
            data_odd = [d[vm.name] for d in violations_odd]
            all_data = data_even + data_odd

            num_profiles = len(all_data)

            data_for_df["vm"].append(vm.name)
            data_for_df["comparison_vm"].append(main_vm.name)
            data_for_df["num_cands"].append(num_candidates)
            data_for_df["num_voters"].append(str((num_voters_even, num_voters_odd)))
            data_for_df["probmod"].append(probmod)            
            data_for_df["num_profiles"].append(num_profiles)

            data_for_df["perc_diff_ws"].append(sum([d["diff_ws"] for d in all_data]) / num_profiles)
            data_for_df["perc_ssw_violation"].append(sum([d["ssw_violation"] for d in all_data]) / num_profiles)
            
            data_with_diff_ws = [d for d in all_data if d["diff_ws"]]
            
            if len(data_with_diff_ws) > 0: 
                data_for_df["perc_ssw_violation_cond_diff_ws"].append(sum([d["ssw_violation"] for d in data_with_diff_ws]) / len(data_with_diff_ws))
            else: 
                data_for_df["perc_ssw_violation_cond_diff_ws"].append(0.0)
                
        df = pd.DataFrame(data_for_df)
        df.to_csv(f"./violations_ssw.csv",index=False)
        


print("done.")
print(df["perc_diff_ws"])



IC
     (4, (10, 11))
     (4, (20, 21))
     (4, (50, 51))
     (4, (100, 101))
     (4, (500, 501))
     (4, (1000, 1001))
     (4, (5000, 5001))


In [14]:
df

,vm,comparison_vm,num_cands,num_voters,num_profiles,probmod,perc_diff_ws,perc_ssw_violation,perc_ssw_violation_cond_diff_ws
0,Beat Path,Split Cycle,4,"(10, 11)",2000,IC,0.0065,0.0065,1.0


## Table 2

Of four-candidate linear profiles in which $BP(\mathbf{P})\neq SV(\mathbf{P})$ find the percentage in which Beat Path violates strong stability for winners with tiebreaking.

In [9]:
df = pd.read_csv("./tables-data-aws/violations_ssw_w_tb.csv")

In [15]:
print("\nOf the profiles in which the Beat Path and Stable Voting winners are different, ")
print("the percent of strong stability for winners with tiebreaking violations for Beat Path (Table 2).")
df_table = df.pivot(index="probmod", columns="num_voters", values="perc_ssw_w_tb_violation_cond_diff_ws")
df_table


Of the profiles in which the Beat Path and Stable Voting winners are different, 
the percent of strong stability for winners with tiebreaking violations for Beat Path (Table 2).


num_voters,"(10, 11)","(100, 101)","(1000, 1001)","(20, 21)","(50, 51)","(500, 501)","(5000, 5001)"
probmod,,,,,,,
IAC,0.268556,0.196979,0.405238,0.213546,0.188899,0.315491,0.592604
IC,0.294130,0.198257,0.222796,0.241378,0.208604,0.208347,0.291578
MALLOWS-RELPHI-R,0.169730,0.103378,0.142149,0.130961,0.108739,0.109586,0.158664
URN-R,0.286654,0.206070,0.378583,0.230778,0.202775,0.305431,0.524672


In [16]:
print("\nThe percent of strong stability for winners with tiebreaking violations for Beat Path.")
df_table = df.pivot(index="probmod", columns="num_voters", values="perc_ssw_w_tb_violation")
df_table


The percent of strong stability for winners with tiebreaking violations for Beat Path.


num_voters,"(10, 11)","(100, 101)","(1000, 1001)","(20, 21)","(50, 51)","(500, 501)","(5000, 5001)"
probmod,,,,,,,
IAC,0.048997,0.007751,0.003304,0.026643,0.012658,0.003752,0.002884
IC,0.061864,0.016612,0.007058,0.039323,0.023482,0.008756,0.004977
MALLOWS-RELPHI-R,0.013395,0.001118,0.000172,0.006011,0.002223,0.000260,0.000038
URN-R,0.025077,0.003439,0.001552,0.012627,0.005619,0.001766,0.001260


In [17]:
print("\nThe percent of profiles in which Beat Path and Stable Voting have different winners.")
df_table = df.pivot(index="probmod", columns="num_voters", values="perc_diff_ws")
df_table


The percent of profiles in which Beat Path and Stable Voting have different winners.


num_voters,"(10, 11)","(100, 101)","(1000, 1001)","(20, 21)","(50, 51)","(500, 501)","(5000, 5001)"
probmod,,,,,,,
IAC,0.182448,0.039352,0.008152,0.124765,0.067009,0.011891,0.004868
IC,0.210329,0.083788,0.031677,0.162911,0.112568,0.042026,0.017067
MALLOWS-RELPHI-R,0.078920,0.010819,0.001210,0.045895,0.020443,0.002368,0.000239
URN-R,0.087484,0.016688,0.004099,0.054715,0.027708,0.005782,0.002401


In [11]:


def has_strong_stability_for_winners_with_tiebreaking_violation(vm, edata, condorcetian_candidates = None, weak_condorcetian_candidates = None): 

    """
    Definition 4.20: Return True if there is a vioalation of strong stability winners with tiebreaking.  
    This function returns True when: 
       
       1.  vm vioaltes stability for winners with tiebreaking for edata, or
       2.  there some candidate is weakly Condorcetian for vm in edata and no candidate is Condorcetian for vm in edata, and
           it is not the case that all candidates who win in edata according to vm are weakly Condorcetian for vm in edata.

    """
    candidates = edata.candidates
    
    if condorcetian_candidates is None: 
        condorcetian_candidates = [c for c in candidates if is_condorcetian(vm, c, edata)]
        
    if weak_condorcetian_candidates is None: 
        weak_condorcetian_candidates = [c for c in candidates if is_weak_condorcetian(vm, c, edata)]
        
    ws = vm(edata)
        
    return (len(condorcetian_candidates) > 0 and not set(ws).issubset(set(condorcetian_candidates)))\
    or (len(weak_condorcetian_candidates) != 0 and len(condorcetian_candidates) == 0 and not set(ws).issubset(set(weak_condorcetian_candidates)))





In [12]:
def record_strong_stability_for_winners_with_tiebreaking_violation(
    vms, 
    main_vm, 
    num_cands, 
    num_voters, 
    probmod, 
    t): 
    
    np.random.seed(current_process().pid)
    prof = generate_profile(num_cands, num_voters, probmod=probmod)
    
    data = {vm.name: dict() for vm in vms}
    for vm in vms: 
        condorcetian_candidates = [c for c in prof.candidates if is_condorcetian(vm, c, prof)]
        weak_condorcetian_candidates = [c for c in prof.candidates if is_weak_condorcetian(vm, c, prof)]
        
        main_ws = main_vm(prof)
        vm_ws = vm(prof)
        data[vm.name] =  {
            "diff_ws": main_ws != vm_ws, 
            "ssw_w_tb_violation": has_strong_stability_for_winners_with_tiebreaking_violation(vm, 
                                                                                               prof, 
                                                                                               condorcetian_candidates = condorcetian_candidates,
                                                                                               weak_condorcetian_candidates = weak_condorcetian_candidates)
        }
    return data


In [13]:
## Parameters for the simulation

# WARNING: The simulation takes a very long time!  
# Set the following to True to skip running the simulation
SKIP_SIMULATION = False  

num_trials = 100#_000

numbers_of_candidates = [
#     3,
    4,
#     5,
#     6,
#     7,
#     8,
#     10, 
#     15, 
#     20
]

numbers_of_voters = [
    (10, 11),
    (20, 21),
    (50, 51),
    (100, 101),
    (500, 501),
    (1000, 1001),
    (5000, 5001),
] 

# uncomment probability models as needed
prob_models = [
    "IC", 
    "IAC", 
    "MALLOWS-RELPHI-R", 
    "URN-R"
]
 
vms = [
    beat_path_Floyd_Warshall,
#     llull,
#     copeland,
#     minimax,
#     uc_gill, # uncovered set
#     instant_runoff,  
#     borda,
#     plurality,
#     getcha
]

main_vm = stable_voting

# use parallel processing to speed up the search
cpus = cpu_count()
print(f'CPUS: {cpus}')
pool = Pool(10) # set to cpus when you can use all the processors

print(f"SKIP_SIMULATION is set to {SKIP_SIMULATION}")

CPUS: 12
SKIP_SIMULATION is set to False


In [14]:
%%time 

data_for_df = {
    "vm": list(),
    "comparison_vm": list(),
    "num_cands": list(), 
    "num_voters": list(),
    "num_profiles": list(),
    "probmod": list(),
    "perc_diff_ws": list(),
    "perc_ssw_w_tb_violation": list(),
    "perc_ssw_w_tb_violation_cond_diff_ws": list()
}

for probmod in prob_models:

    print(probmod)

    voting_scenarios = list(product(numbers_of_candidates, numbers_of_voters))

    data = {vm.name: {_vs: {"total_profiles": 0, 
                            "total_even": list(),
                            "total_odd": list()} 
                      for _vs in voting_scenarios} 
            for vm in vms}

    for vs in voting_scenarios:
        print(f"     {vs}")
        num_candidates = vs[0]
        num_voters_even = vs[1][0]
        num_voters_odd = vs[1][1]

        # find the violations in parallel

        record_violations_even = partial(
            record_strong_stability_for_winners_with_tiebreaking_violation,
            vms, 
            main_vm, 
            num_candidates, 
            num_voters_even, 
            probmod)

        record_violations_odd = partial(
            record_strong_stability_for_winners_with_tiebreaking_violation,
            vms, 
            main_vm, 
            num_candidates, 
            num_voters_odd, 
            probmod)

        violations_even = pool.map(record_violations_even, range(num_trials))
        violations_odd = pool.map(record_violations_odd, range(num_trials))

        # done calculating the violations
        for vm in vms:
            data_even = [d[vm.name] for d in violations_even]
            data_odd = [d[vm.name] for d in violations_odd]
            all_data = data_even + data_odd
            num_profiles = len(all_data)

            data_for_df["vm"].append(vm.name)
            data_for_df["comparison_vm"].append(main_vm.name)
            data_for_df["num_cands"].append(num_candidates)
            data_for_df["num_voters"].append(str((num_voters_even, num_voters_odd)))
            data_for_df["num_profiles"].append(num_profiles)
            data_for_df["probmod"].append(probmod)
            data_for_df["perc_diff_ws"].append(sum([d["diff_ws"] for d in all_data]) / num_profiles)
            data_for_df["perc_ssw_w_tb_violation"].append(sum([d["ssw_w_tb_violation"] for d in all_data]) / num_profiles)
            
            data_with_diff_ws = [d for d in all_data if d["diff_ws"]]
            
            if len(data_with_diff_ws) > 0: 
                data_for_df["perc_ssw_w_tb_violation_cond_diff_ws"].append(sum([d["ssw_w_tb_violation"] for d in data_with_diff_ws]) / len(data_with_diff_ws))
            else: 
                data_for_df["perc_ssw_w_tb_violation_cond_diff_ws"].append(0.0)
                
        df = pd.DataFrame(data_for_df)
        df.to_csv(f"./violations_ssw_w_tb.csv",index=False)
        
print("done.")


IC
     (4, (10, 11))
     (4, (20, 21))
     (4, (50, 51))
     (4, (100, 101))
     (4, (500, 501))
     (4, (1000, 1001))
     (4, (5000, 5001))
IAC
     (4, (10, 11))
     (4, (20, 21))
     (4, (50, 51))
     (4, (100, 101))
     (4, (500, 501))
     (4, (1000, 1001))
     (4, (5000, 5001))
MALLOWS-RELPHI-R
     (4, (10, 11))
     (4, (20, 21))
     (4, (50, 51))
     (4, (100, 101))
     (4, (500, 501))
     (4, (1000, 1001))
     (4, (5000, 5001))
URN-R
     (4, (10, 11))
     (4, (20, 21))
     (4, (50, 51))
     (4, (100, 101))
     (4, (500, 501))
     (4, (1000, 1001))
     (4, (5000, 5001))
done.
CPU times: user 1.37 s, sys: 280 ms, total: 1.65 s
Wall time: 2.36 s


In [ ]:
df

## Table 3

Estimated average sizes of winning sets for profiles with a given number of candidates in the limit as the number of voters goes to infinity.

In [3]:
df = pd.read_csv("./tables-data/irresoluteness_inf_voters.csv")

In [4]:
print("\n Average Winning Set Size (Table 3)\n")
df_table = df.pivot(index="vm", columns="num_cands", values="avg_ws_size")
df_table


 Average Winning Set Size (Table 3)



num_cands,3,4,5,6,7,8,9,10,20,30
vm,,,,,,,,,,
Copeland,1.17546,1.261423,1.290715,1.301886,1.307802,1.312268,1.314079,1.310383,1.278140,1.246339
GETCHA,1.17546,1.442093,1.790295,2.201078,2.707892,3.301104,3.964455,4.649134,13.502768,22.881436
Split Cycle,1.00000,1.012167,1.033383,1.055462,1.082580,1.108534,1.138317,1.162874,1.417532,1.623496
Uncovered Set,1.17546,1.351758,1.532496,1.707575,1.891731,2.082329,2.276663,2.457021,4.544439,6.831056


In [5]:
print("\n Proportion with Multiple Winners\n")
df_table = df.pivot(index="vm", columns="num_cands", values="perc_mult_winners")
df_table


 Proportion with Multiple Winners



num_cands,3,4,5,6,7,8,9,10,20,30
vm,,,,,,,,,,
Copeland,0.08773,0.175879,0.218381,0.234677,0.241348,0.245658,0.249328,0.246231,0.227297,0.206776
GETCHA,0.08773,0.175879,0.252376,0.314116,0.367102,0.415331,0.456046,0.485792,0.680487,0.763655
Split Cycle,0.00000,0.012167,0.032639,0.053195,0.078056,0.100531,0.126751,0.146315,0.324321,0.431576
Uncovered Set,0.08773,0.175879,0.252376,0.314116,0.367102,0.415331,0.456046,0.485792,0.680487,0.763655


In [6]:
print("\n Proportion of Profiles with a Condorcet Winner\n")
df_table = df.pivot(index="vm", columns="num_cands", values="perc_condorcet_winner")
df_table


 Proportion of Profiles with a Condorcet Winner



num_cands,3,4,5,6,7,8,9,10,20,30
vm,,,,,,,,,,
Copeland,0.91227,0.824121,0.747624,0.685884,0.632898,0.584669,0.543954,0.514208,0.319513,0.236345
GETCHA,0.91227,0.824121,0.747624,0.685884,0.632898,0.584669,0.543954,0.514208,0.319513,0.236345
Split Cycle,0.91227,0.824121,0.747624,0.685884,0.632898,0.584669,0.543954,0.514208,0.319513,0.236345
Uncovered Set,0.91227,0.824121,0.747624,0.685884,0.632898,0.584669,0.543954,0.514208,0.319513,0.236345


In [31]:
def record_ws_data(vms, num_cand, t): 
    np.random.seed(current_process().pid)
    mg = generate_edge_ordered_tournament_infinite_limit(num_cand)
    #winning_sets = {vm.name: vm(mg) for vm in vms}
    winning_sets = {vm.name: [0] for vm in vms}
    return {
        "has_cw": True, #mg.condorcet_winner() is not None,
        "winning_set_sizes": {vm.name: len(winning_sets[vm.name]) for vm in vms}
        }


In [34]:
all_num_cands = [
        3,
        4,
        5,
        6,
        7,
        8,
        9, 
        10,
        20,
        30
    ]

vms = [
    split_cycle, 
    copeland, 
    uc_gill,
    getcha
]
num_cpus = 10
num_trials = 5

In [35]:
%%time

data_for_df = {
    "num_cands": list(), 
    "num_margin_graphs": list(),
    "vm": list(),
    "perc_mult_winners": list(),
    "avg_ws_size": list(),
    "perc_condorcet_winner": list()
    }
    
    
for num_cands in all_num_cands: 
    print(num_cands)
        
    get_ws_data= partial(record_ws_data,
                         vms,
                         num_cands)

    pool = Pool(num_cpus) # set to cpus when you can use all the processors
    total_ws_data = pool.map(get_ws_data, range(num_trials))
        
    for vm in vms: 
        data = [d["winning_set_sizes"][vm.name] for d in total_ws_data]
        data_for_df["num_cands"].append(num_cands)
        data_for_df["num_margin_graphs"].append(len(data))
        data_for_df["vm"].append(vm.name)
        data_for_df["perc_mult_winners"].append(sum([d > 1 for d in data]) / len(data) if len(data) > 0 else np.nan)
        data_for_df["avg_ws_size"].append(np.average([d for d in data]))
        data_for_df["perc_condorcet_winner"].append(sum([d["has_cw"]  for d in total_ws_data]) / len(total_ws_data))
            
    df = pd.DataFrame(data_for_df)
    df.to_csv(f"./irresoluteness_inf_voters.csv",index=False)


3
4
5
6
7
8
9
10
20
30
CPU times: user 158 ms, sys: 345 ms, total: 503 ms
Wall time: 1min 59s


In [36]:
for _ in range(10): 
    record_ws_data(vms, 3, 0)

In [10]:
def record_ws_data(vms, num_cand, t): 
    np.random.seed(current_process().pid)
    mg = generate_edge_ordered_tournament_infinite_limit(num_cand)
    winning_sets = {vm.name: vm(mg) for vm in vms}
    return {
        "has_cw": mg.condorcet_winner() is not None,
        "winning_set_sizes": {vm.name: len(winning_sets[vm.name]) for vm in vms}
        }

def find_irresoluteness_data_inf_voters(
    vms, 
    num_trials = 10, 
    all_num_cands = [
        3,
        4,
        5,
        6,
        7,
        8,
        9, 
        10,
        20,
        30
    ],
    num_cpus = 10
):
    data_for_df = {
        "num_cands": list(), 
        "num_margin_graphs": list(),
        "vm": list(),
        "perc_mult_winners": list(),
        "avg_ws_size": list(),
        "perc_condorcet_winner": list()
    }
    
    
    for num_cands in tqdm(all_num_cands): 
        print(num_cands)
        
        get_ws_data= partial(record_ws_data,
                             vms,
                             num_cands)

        pool = Pool(num_cpus) # set to cpus when you can use all the processors
        total_ws_data = pool.map(get_ws_data, range(num_trials))
        
        for vm in vms: 
            data = [d["winning_set_sizes"][vm.name] for d in total_ws_data]
            data_for_df["num_cands"].append(num_cands)
            data_for_df["num_margin_graphs"].append(len(data))
            data_for_df["vm"].append(vm.name)
            data_for_df["perc_mult_winners"].append(sum([d > 1 
                                                         for d in data]) / len(data) 
                                                    if len(data) > 0 else np.nan)
            data_for_df["avg_ws_size"].append(np.average([d for d in data]))
            data_for_df["perc_condorcet_winner"].append(sum([d["has_cw"]  for d in total_ws_data]) / len(total_ws_data))
            
        df = pd.DataFrame(data_for_df)
        df.to_csv(f"./irresoluteness_inf_voters.csv",index=False)
            
    
    print("done.")
    return df

In [11]:
%%time


    
df = find_irresoluteness_data_inf_voters(
    vms, 
    num_trials = num_trials, 
    all_num_cands = all_num_cands,
    num_cpus = num_cpus
)

  0%|          | 0/10 [00:00<?, ?it/s]

3
4
5
6
7
8
9
10
20


In [1]:
def record_ws_data(vms, num_cand, t): 
    np.random.seed(current_process().pid)
    mg = generate_edge_ordered_tournament_infinite_limit(num_cand)
    winning_sets = {vm.name: vm(mg) for vm in vms}
    return {
        "has_cw": mg.condorcet_winner() is not None,
        "winning_set_sizes": {vm.name: len(winning_sets[vm.name]) for vm in vms}
        }


In [9]:
 
%%time 


vms = [
    split_cycle, 
    copeland, 
    uc_gill,
    getcha
]

for _ in range(10):
    np.random.seed(current_process().pid)
    mg = generate_edge_ordered_tournament_infinite_limit(30)

    for vm in vms: 
        vm(mg)
    mg.condorcet_winner()

CPU times: user 20.2 s, sys: 54.4 s, total: 1min 14s
Wall time: 7.2 s


In [26]:
df

,num_cands,num_margin_graphs,vm,perc_mult_winners,avg_ws_size,perc_condorcet_winner
0,3,10,Split Cycle,0.0,1.0,1.0
1,3,10,Copeland,0.0,1.0,1.0
2,3,10,Uncovered Set,0.0,1.0,1.0
3,3,10,GETCHA,0.0,1.0,1.0
4,4,10,Split Cycle,0.0,1.0,0.8
5,4,10,Copeland,0.2,1.3,0.8
6,4,10,Uncovered Set,0.2,1.4,0.8
7,4,10,GETCHA,0.2,1.5,0.8
8,5,10,Split Cycle,0.0,1.0,0.7
9,5,10,Copeland,0.3,1.4,0.7


In [13]:
df = pd.read_csv("./tables-data/irresoluteness_inf_voters.csv")

In [14]:
df

,num_cands,num_margin_graphs,vm,perc_mult_winners,avg_ws_size,perc_condorcet_winner
0,3,1000000,Split Cycle,0.000000,1.000000,0.912270
1,3,1000000,Copeland,0.087730,1.175460,0.912270
2,3,1000000,Uncovered Set,0.087730,1.175460,0.912270
3,3,1000000,GETCHA,0.087730,1.175460,0.912270
4,4,1000000,Split Cycle,0.012167,1.012167,0.824121
5,4,1000000,Copeland,0.175879,1.261423,0.824121
6,4,1000000,Uncovered Set,0.175879,1.351758,0.824121
7,4,1000000,GETCHA,0.175879,1.442093,0.824121
8,5,1000000,Split Cycle,0.032639,1.033383,0.747624
9,5,1000000,Copeland,0.218381,1.290715,0.747624
